In [1]:
import sys
import os

# Add the project root to the path
if os.path.abspath('.') not in sys.path:
    sys.path.append(os.path.abspath('.'))

# Import the setup module
from notebook_setup import setup_django
setup_django()

# Now you can import Django models
from common.models import Brokers, Prices, Transactions, FX, Assets
from users.models import CustomUser
from datetime import date, datetime, timedelta
from django.db.models import F, Sum

Django initialized successfully!


In [2]:
selected_brokers = broker_ids = [2]
effective_current_date = date.today()
currency_target = 'USD'
number_of_digits = 2

CustomUser.objects.get(id=2)

<CustomUser: Yaroslav>

In [3]:
security_id = 7
security = Assets.objects.get(id=security_id)
security.price_at_date(effective_current_date).price

# for field in FX._meta.get_fields():
#     print(field.name)

quote = security.prices.filter(date__lte=effective_current_date).order_by('-date').first()
print(currency_target)
# if currency_target is not None:
#     quote.price = quote.price * FX.get_rate(security.currency, currency_target, effective_current_date, investor=2)['FX']
# quote

FX.get_rate('USD', 'GBP', effective_current_date)

USD


{'FX': Decimal('0.747200'),
 'conversions': 1,
 'dates_async': False,
 'dates': [datetime.date(2024, 9, 30)]}

In [4]:
from django.db.models import Count

duplicates = (
            Prices.objects.values('date', 'security', 'price')
            .annotate(count_id=Count('id'))
            .filter(count_id__gt=1)
        )

total_duplicates = len(duplicates)
print((f"Found {total_duplicates} duplicate entries."))

for i, duplicate in enumerate(duplicates, start=1):
            # Keep one entry and delete the rest
            entries = Prices.objects.filter(
                date=duplicate['date'],
                security=duplicate['security'],
                price=duplicate['price']
            )
            entries_to_keep = entries.first()
            entries.exclude(id=entries_to_keep.id).delete()

            # Print status update
            print(f"Duplicates are being removed: {i} out of {total_duplicates}\r", end='', flush=True)
            
print("\nDuplicates removed.")


Found 0 duplicate entries.

Duplicates removed.


In [5]:
import datetime
from decimal import Decimal

from portfolio_management.core.portfolio_utils import NAV_at_date


def end_of_year_price_correction(user, year, broker_name, target_nav, asset_name):
    
    target_nav = round(Decimal(target_nav), 2)
    
    # Get the broker
    try:
        broker = Brokers.objects.get(name=broker_name)
    except Brokers.DoesNotExist:
        return {"error": f"Broker {broker_name} does not exist."}

    # Get the asset
    try:
        asset = Assets.objects.get(name=asset_name)
    except Assets.DoesNotExist:
        return {"error": f"Asset {asset_name} does not exist."}

    # Calculate end of year date
    end_of_year_date = datetime.date(year, 12, 31)

    # Fetch NAV at the end of the year
    # nav_at_end_of_year = get_nav_at_date(broker, end_of_year_date)
    nav_at_end_of_year = NAV_at_date(user.id, [broker.id], end_of_year_date, asset.currency, [])['Total NAV']
    if nav_at_end_of_year is None:
        return {"error": f"No NAV found for broker {broker_name} at the end of {year}."}

    # Fetch asset price and position at the end of the year
    price_at_end_of_year = asset.price_at_date(end_of_year_date)
    if not price_at_end_of_year:
        return {"error": f"No price found for asset {asset_name} at the end of {year}."}

    position_at_end_of_year = asset.position(end_of_year_date, user, [broker.id])

    # Calculate new price
    old_price = price_at_end_of_year.price
    new_price = old_price + ((target_nav - nav_at_end_of_year) / position_at_end_of_year)

    # Display information
    old_asset_value = round(Decimal(old_price * position_at_end_of_year), 2)
    new_asset_value = round(Decimal(new_price * position_at_end_of_year), 2)

    result = {
        "old_price": old_price,
        "new_price": new_price,
        "old_asset_value": old_asset_value,
        "new_asset_value": new_asset_value,
        "old_nav": nav_at_end_of_year,
        "target_nav": target_nav
    }

    print(f"Old Price: {result['old_price']}, New Price: {result['new_price']}")
    print(f"Old Asset Value: {result['old_asset_value']}, New Asset Value: {result['new_asset_value']}")
    print(f"Old NAV: {result['old_nav']}, Target NAV: {result['target_nav']}")

    # Ask for confirmation
    confirm = input("Do you want to update the price? (yes/no): ")

    if confirm.lower() == 'yes':
        # Update the price
        price_instance, created = Prices.objects.get_or_create(
            security=asset, date=end_of_year_date,
            defaults={'price': new_price}
        )
        if not created:
            price_instance.price = new_price
            price_instance.save()

        result["status"] = "Price updated successfully."
        print("New price is:", price_instance.price)
    else:
        result["status"] = "Price update canceled."

    return result

user = CustomUser.objects.filter(username='Yaroslav').first()
year = 2013
broker_name = 'UBS Pension'
target_nav = 29444.38
asset_name = 'Emerging Markets Equity Fund'

end_of_year_price_correction(user, year, broker_name, target_nav, asset_name)

ModuleNotFoundError: No module named 'portfolio_management.core'

In [6]:
from common.models import Accounts

user = CustomUser.objects.get(username='Y')

a = Accounts.objects.get(broker__investor=user, name='Test broker')
a.transactions.filter(quantity__isnull=False)

s = Assets.objects.filter(transactions__broker_account=a, transactions__date__lte=date(2020,6,1))
s.distinct().count()


ValueError: Cannot query "Account: Test broker": Must be "Transactions" instance.

In [10]:
async def async_generator():
    yield "Initial async value from generator"
    # Process the received value
    print(f"Sent initial value")
    
    for i in range(3):    
        
        response =yield f'Sending {i}'

        print(f"Response from consumer: {response}")

async def async_consumer():
    gen = async_generator()
    
    # Start the async generator
    result = await gen.asend(None)  # Pass None on the first call
    print(f"Consumer received: {result}")

    async for generator_tick in gen:
        print(f"Received from generator: {generator_tick}")
    
        await gen.asend(f"Message from consumer")
        # print(f"Consumer received: {response}")
    
    # Close the async generator
    await gen.aclose()


await async_consumer()

Consumer received: Initial async value from generator
Sent initial value
Received from generator: Sending 0
Response from consumer: Message from consumer
Response from consumer: None
Received from generator: Sending 2
Response from consumer: Message from consumer


StopAsyncIteration: 

In [21]:
from tinkoff.invest import Client

from users.models import TinkoffApiToken

user = CustomUser.objects.get(id=1)

TM = TinkoffApiToken.objects.get(user=user, token_type='read_only', sandbox_mode=False, is_active=True)
token = TM.get_token(user=user)

with Client(token) as client:
    accounts = client.users.get_accounts()
    print(accounts)


GetAccountsResponse(accounts=[Account(id='2016485761', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='Брокерский счёт', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2019, 12, 21, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>), Account(id='2145117784', type=<AccountType.ACCOUNT_TYPE_TINKOFF_IIS: 2>, name='ИИС', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2024, 3, 18, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_READ_ONLY: 2>), Account(id='2204772891', type=<AccountType.ACCOUNT_TYPE_INVEST_BOX: 3>, name='Инвесткопилка', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2024, 11, 11, 0, 0, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1

In [25]:
from tinkoff.invest import Client, OperationState, OperationType, GetOperationsByCursorRequest, InstrumentType
from tinkoff.invest.utils import now, quotation_to_decimal
from datetime import timedelta

from constants import OPERATION_TYPE_DESCRIPTIONS, INSTRUMENT_KIND_DESCRIPTIONS
from users.models import TinkoffApiToken

user = CustomUser.objects.get(id=1)

def get_recent_operations(user, days_back=30):
    # Fixed token retrieval method using TinkoffApiToken directly
    TM = TinkoffApiToken.objects.get(user=user, token_type='read_only', sandbox_mode=False, is_active=True)
    token = TM.get_token(user=user)
    
    with Client(token) as client:
        accounts = client.users.get_accounts()
        if not accounts.accounts:
            print("No accounts found.")
            return
        
        # If there's more than one account, let the user choose
        if len(accounts.accounts) > 1:
            print("Available accounts:")
            for i, account in enumerate(accounts.accounts, 1):
                print(f"{i}. {account.name} (ID: {account.id})")
            
            while True:
                try:
                    choice = int(input("Select an account number: ")) - 1
                    if 0 <= choice < len(accounts.accounts):
                        selected_account = accounts.accounts[choice]
                        break
                    else:
                        print("Invalid selection. Please try again.")
                except ValueError:
                    print("Please enter a valid number.")
        else:
            selected_account = accounts.accounts[0]
        
        print(f"\nFetching operations for account: {selected_account.name}")
        
        # Set the time range for operations
        to_date = now()
        from_date = to_date - timedelta(days=days_back)
        
        # Initialize cursor and operations list
        cursor = ""
        all_operations = []
        
        while True:
            # Request operations
            response = client.operations.get_operations_by_cursor(
                GetOperationsByCursorRequest(
                    account_id=selected_account.id,
                    from_=from_date,
                    to=to_date,
                    cursor=cursor,
                    limit=1000,  # Adjust this value as needed
                    operation_types=[],  # Empty list means all types
                    state=OperationState.OPERATION_STATE_EXECUTED
                )
            )
            
            all_operations.extend(response.items)
            
            if not response.has_next:
                break
            
            cursor = response.next_cursor
        
        # Process and display operations
        if not all_operations:
            print("No operations found in the specified time range.")
            return
        
        print(f"\nRecent operations (last {days_back} days):")
        for op in all_operations:
            operation_date = op.date.strftime("%Y-%m-%d %H:%M:%S")
            operation_type = OPERATION_TYPE_DESCRIPTIONS.get(op.type, "Unknown Operation Type")
            instrument_name = op.name
            instrument_kind = INSTRUMENT_KIND_DESCRIPTIONS.get(op.instrument_kind, "Unknown Instrument Type")
            quantity = op.quantity
            price = quotation_to_decimal(op.price) if op.price else "N/A"
            currency = op.payment.currency if op.payment else "N/A"
            payment = quotation_to_decimal(op.payment) if op.payment else "N/A"

            if operation_type == 'Buy Securities':
                print("-------------")
                print(op)
                print("-------------")
            
            # print(f"Date: {operation_date}")
            # print(f"Type: {operation_type}")
            # print(f"Instrument: {instrument_name}")
            # print(f"Description: {op.description}")
            # print(f"Instrument type: {op.instrument_type}")
            # print(f"Instrument kind: {instrument_kind}")
            # print(f"Quantity: {quantity}")
            # print(f"Price: {price} {currency}")
            # print(f"Total Payment: {payment} {currency}")
            # print(f"Trades list: {op.trades_info}")
            # print("---")

# Example usage:
get_recent_operations(user, days_back=30) 

Available accounts:
1. Брокерский счёт (ID: 2016485761)
2. ИИС (ID: 2145117784)
3. Инвесткопилка (ID: 2204772891)
4. Накопительный счет плюс (ID: 2204775639)

Fetching operations for account: Брокерский счёт

Recent operations (last 30 days):
-------------
OperationItem(cursor='1742553477478309351', broker_account_id='2016485761', id='1984992764470998309', parent_operation_id='', name='MIX-6.25 Индекс МосБиржи', date=datetime.datetime(2025, 3, 21, 10, 37, 57, 478000, tzinfo=datetime.timezone.utc), type=<OperationType.OPERATION_TYPE_BUY: 15>, description='Покупка 1 фьючерса MIX-6.25 Индекс МосБиржи', state=<OperationState.OPERATION_STATE_EXECUTED: 1>, instrument_uid='00cd9005-fded-495f-b46b-c0d501e030f7', figi='FUTMIX062500', instrument_type='futures', instrument_kind=<InstrumentType.INSTRUMENT_TYPE_FUTURES: 5>, position_uid='1fa5a66e-fbf3-47cc-b2b4-2ba96658c79f', payment=MoneyValue(currency='rub', units=-338675, nano=0), price=MoneyValue(currency='pt.', units=338675, nano=0), commissio

In [1]:
broker = Brokers.objects.get(name='Tinkoff')

for b in Brokers.objects.all():
    print(b.name, b.id)

# for i in broker.tinkoff_tokens.all():
#     print(i)

broker.tinkoff_tokens.exists()

NameError: name 'Brokers' is not defined